In [1]:
from scipy.special import softmax
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix, csr_matrix, issparse, diags

from scipy.sparse import lil_matrix
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import time

from numpy.linalg import solve

import heapq
from collections import defaultdict


In [2]:
def extract_state_action_pairs(outgoing_dict):
    row_idx, col_idx = [], []
    for s, actions in outgoing_dict.items():
        row_idx.extend([s] * len(actions))
        col_idx.extend(actions)
    return np.array(row_idx), np.array(col_idx)


def compute_softmax_policy_by_action_features(action_features, theta, n_links, row_idx, col_idx):

    vals = sum(theta[i] * f[col_idx] for i, f in enumerate(action_features))


    sort_idx = np.argsort(row_idx)
    row_idx_sorted = row_idx[sort_idx]
    col_idx_sorted = col_idx[sort_idx]
    vals_sorted = vals[sort_idx]


    pi_data = np.zeros_like(vals_sorted)
    i = 0
    while i < len(row_idx_sorted):
        s = row_idx_sorted[i]
        j = i
        while j < len(row_idx_sorted) and row_idx_sorted[j] == s:
            j += 1
        scores = vals_sorted[i:j]
        exp_scores = np.exp(scores - np.max(scores))  # for numerical stability
        probs = exp_scores / np.sum(exp_scores)
        pi_data[i:j] = probs
        i = j


    pi_sparse = coo_matrix((pi_data, (row_idx_sorted, col_idx_sorted)), shape=(n_links, n_links)).tocsr()
    return pi_sparse




        
def build_P_matrix(Q, m, Pi, outgoing_dict, row_idx, col_idx):

    n = Q.shape[0]
    P = m[:, None] * Q  # baseline: m_s * q_su  correct!
    
    #Pi_dense = Pi.toarray()
    #delta = (1 - m[row_idx]) * Pi_dense[row_idx, col_idx]
    
    delta = (1 - m[row_idx]) * Pi[row_idx, col_idx].A1  # .A1 makes it 1D

    P[row_idx, col_idx] += delta


    return P



def compute_kappa(v, c, q, beta):
    return np.sum((v - beta * c) * q, axis=1)

def compute_R(mu, tau, omega, m, kappa, beta):
    num = np.sum(mu * (-beta * tau + kappa * m))
    denom = np.sum(mu * (tau + omega * m))
    return num / denom if denom > 0 else -np.inf

def compute_expected_time(Q, c):
    #return np.array(Q.multiply(c).sum(axis=1)).flatten()
    return np.sum(Q * c, axis=1)

def smooth_zero_entries(mat, eps=1e-8):
    mat = mat.copy()
    n, d = mat.shape

    for i in range(n):
        row = mat[i]
        row_sum = np.sum(row)

        if row_sum == 0:
            continue  # skip zero rows

        zero_mask = (row == 0)
        nonzero_mask = (row > 0)

        num_zeros = np.sum(zero_mask)
        if num_zeros == 0:
            continue  

        mat[i, zero_mask] = eps

        nonzero_sum = np.sum(row[nonzero_mask])
        adjusted_total = row_sum - num_zeros * eps

        if nonzero_sum > 0:
            mat[i, nonzero_mask] = row[nonzero_mask] * (adjusted_total / nonzero_sum)

    return mat

In [3]:
def compute_joint_fixed_point_single_loop(
    lambda_vec, tau, omega, M, alpha, Q, outgoing_dict, pi, init_mu, init_m, row_idx, col_idx,v,c,
    tol_m=1e-3, tol_mu=1e-4, max_iter=2000, return_history=False
):

    import time
    n = len(lambda_vec)
    
    m = init_m     # initial matching probability
    mu = init_mu
    #print(f'initial m:{m}')
    #print(f'iniital mu:{mu}')
    

    m_history, mu_history = [], []
    t0 = time.time()
    for outer in range(max_iter):
        #print(f'\nIteration {outer}')
        t1 = time.time()

        ### Step 1: update m (1 step)
        m_old = m.copy()
        
        weighted_sum = np.dot(mu, tau + m * omega)
        safe_mu = np.where(mu > 0, mu, 1.0)
        exponent = -alpha * lambda_vec * weighted_sum / (M * safe_mu)
        m = 1 - np.exp(exponent)
        m = np.where(mu > 0, m, 1.0)
        a = (outer)/(outer+1) 
        m_diff = np.max(np.abs(m - m_old))
        
        
        #m_rdiff = np.max(np.abs((m - m_old)/m_old))
        #a = 0.4
        m = a * m + (1-a) * m_old
        #m = m_old + 1/(outer+1) * (m - m_old)
        # = 1/(outer+1) * m  + outer/(outer+1) * m_old
        
        t2 = time.time()
        #print('m time:', t2-t1)

        #m_relax_diff = np.max(np.abs(m - m_old))


        ### Step 2: build P
        P = build_P_matrix(Q, m, pi, outgoing_dict, row_idx, col_idx)
        
        t3= time.time()

        #print('P time:', t3-t2)
        ### Step 3: update mu (1 step)
        mu_old = mu.copy()
        mu = mu_old @ P
        mu_diff = np.max(np.abs(mu - mu_old))
        mu_norm = 0.5 * np.sum(np.abs(mu - mu_old))

        mu = a*mu + (1-a)*mu_old
        #mu = mu_old + 1/(outer+1) * (mu - mu_old)


        t4 = time.time()
        #print('mu time:', t4-t3)

        ### Print iteration info
        m_history.append(m_diff)
        mu_history.append(mu_norm)

        #print(f'ite:{outer}, m_diff: {m_diff:.4e}, mu_norm: {mu_norm:.4e}, mu_diff:{mu_diff:.4e}')

        h = lambda_vec * np.sum(mu*omega) / (M * mu)
        


        if m_diff < tol_m and mu_norm < tol_mu:
        #if outer == 4999:
            t3 = time.time()
            print(f'Converged! Iteration:{outer}, Duration:{t3-t0}')
            print(f'm_diff: {m_diff:.4e},  mu_norm: {mu_norm:.4e}, mu_diff:{mu_diff:.4e}')

            print(f'min m:{np.min(m)}, 0.5:{np.sum(m>0.5)},0.8:{np.sum(m>0.8)}, 0.9:{np.sum(m>0.9)},0.99:{np.sum(m>0.99)}')
            #print('final mu:', mu)
            #print(':',pi)
            break
        outer_array = [0,500,1000,1500, 2000, 2500,2999,3500,4000,4500,4999]
        if outer in outer_array:
            kappa = compute_kappa(v, c, Q, beta)
            R_pi = compute_R(mu, tau, omega, m, kappa, beta)
            #print(f'ite:{outer},R_pi:{-R_pi}')
            #print(f'Not converged.')
            print(f'ite:{outer},m_diff: {m_diff:.4e},  mu_norm: {mu_norm:.4e}, mu_diff:{mu_diff:.4e}, R:{R_pi}')



    if return_history:
        return mu, m, m_history, mu_history
    return mu, m


In [4]:
### Derivative-free

def make_wrapped_objective(
    outgoing_dict,
    action_features,
    lambda_vec,
    tau,
    omega,
    M,
    alpha,
    Q,
    v,
    c,
    beta,
    init_mu
):
    mu_last = init_mu.copy()  
    m_last = np.full(n, 0.2, dtype=np.float32) 
    theta_hist = []
    R_hist = []
    m_hist = []
    mu_hist = []

    

    def objective_wrapper(theta_vec):
        nonlocal mu_last, m_last  
        print(f'\ntheta:{theta_vec}')

        n_links = len(lambda_vec)
        row_idx, col_idx = extract_state_action_pairs(outgoing_dict)

        #  policy
        pi = compute_softmax_policy_by_action_features(
            action_features, theta_vec, n_links, row_idx, col_idx
        )


        mu, m = compute_joint_fixed_point_single_loop(
            lambda_vec, tau, omega, M, alpha, Q, outgoing_dict, pi,
            mu_last, m_last, row_idx, col_idx, v, c,
            tol_m=1e-3, tol_mu=1e-4, max_iter=20000, return_history=False
        )


        kappa = compute_kappa(v, c, Q, beta)
        R_pi = compute_R(mu, tau, omega, m, kappa, beta)

        #  warm start 
        #mu_last = mu.copy()
        #m_last = m.copy()


        theta_hist.append(theta_vec.copy())
        R_hist.append(R_pi)
        m_hist.append(m.copy())
        mu_hist.append(mu.copy())


        theta_str = ", ".join([f"{t:.6f}" for t in theta_vec])
        print(f"[Step] theta = [{theta_str}], R(pi) = {R_pi:.6f}")

        return -R_pi  # minimize -R(pi)


    return objective_wrapper, theta_hist, R_hist, m_hist, mu_hist


In [5]:
### Build network

def allpairs_shortest_time_on_nodes(n_nodes, adjacency):  

    INF = 1e18
    dist = np.full((n_nodes, n_nodes), INF, dtype=float)
    for src in range(n_nodes):
        d = np.full(n_nodes, INF, dtype=float)
        d[src] = 0.0
        pq = [(0.0, src)]
        while pq:
            du, u = heapq.heappop(pq)
            if du != d[u]: 
                continue
            for v, w in adjacency[u]:
                nd = du + w
                if nd < d[v]:
                    d[v] = nd
                    heapq.heappush(pq, (nd, v))
        dist[src, :] = d
    return dist

def build_dense_link_to_link_c(directed_links, time_min_per_link, n_nodes, include_linkj_travel=True):


    adjacency = [[] for _ in range(n_nodes)]
    for (u, v), w in zip(directed_links, time_min_per_link):
        adjacency[u].append((v, w))  


    node_dist = allpairs_shortest_time_on_nodes(n_nodes, adjacency)  


    L = len(directed_links)
    tail_of = np.array([u for (u, v) in directed_links])
    head_of = np.array([v for (u, v) in directed_links])
    c = np.zeros((L, L), dtype=float)

    add_j = time_min_per_link if include_linkj_travel else 0.0
    for i in range(L):
        vi = head_of[i]                 

        c[i, :] = node_dist[vi, tail_of] + add_j

    return c  




def build_sioux_falls_like(seed=42,
                           grid_w=6, grid_h=4,
                           speed_mph=40.0,          # 与 c/40 一致
                           initial_charge=35.0,
                           unit_fare=35.0,
                           alpha=0.8,
                           M=10000,                 
                           b=0.2):

    rng = np.random.default_rng(seed)


    def nid(r, c):
        return r * grid_w + c

    coords = {}
    for r in range(grid_h):
        for c in range(grid_w):
 
            coords[nid(r,c)] = (c + 0.05*rng.normal(), r + 0.05*rng.normal())

    undirected_edges = []

    for r in range(grid_h):
        for c in range(grid_w-1):
            undirected_edges.append((nid(r,c), nid(r,c+1)))

    for r in range(grid_h-1):
        for c in range(grid_w):
            undirected_edges.append((nid(r,c), nid(r+1,c)))


    directed_links = []   # (u -> v)
    for u, v in undirected_edges:
        directed_links.append((u, v))
        directed_links.append((v, u))
    L = len(directed_links)   # 76


    def euclid(a, b):
        ax, ay = coords[a]
        bx, by = coords[b]
        return np.hypot(ax-bx, ay-by)

    link_length = np.array([euclid(u, v) for (u, v) in directed_links])  # (L,)

    link_length *= (1.0 + 0.05*rng.normal(size=L))
    link_length = np.maximum(link_length, 1e-3)


    time_min_per_link = 60.0 * link_length / speed_mph



    head_of = np.array([v for (_, v) in directed_links])
    tail_of = np.array([u for (u, _) in directed_links])


    node_to_out_links = defaultdict(list)
    for j, (u, v) in enumerate(directed_links):
        node_to_out_links[v].append(j)

    outgoing_dict = {}
    for i, (u, v) in enumerate(directed_links):
        outgoing = node_to_out_links[v]

        outgoing_dict[i] = outgoing.copy()

    outgoing_links = outgoing_dict  


    n_nodes = grid_w * grid_h
    c = build_dense_link_to_link_c(
            directed_links=directed_links,
            time_min_per_link=time_min_per_link,
            n_nodes=n_nodes,
            include_linkj_travel=True  
    )
    c_time = c / 40.0  


    cbd_rows = range(max(0, grid_h//2 - 1), min(grid_h, grid_h//2 + 1))
    cbd_cols = range(max(0, grid_w//2 - 1), min(grid_w, grid_w//2 + 1))
    cbd_nodes = {nid(r, c) for r in cbd_rows for c in cbd_cols}


    is_cbd_link = np.array([(u in cbd_nodes) or (v in cbd_nodes) for (u, v) in directed_links])


    tau = np.where(is_cbd_link, 0.05, 0.20)  



    dest = np.zeros((L, L), dtype=float)

    cbd_sink_mask = np.array([head in cbd_nodes for (_, head) in directed_links])
    non_cbd_sink_mask = ~cbd_sink_mask


    for i in range(L):
        if is_cbd_link[i]:
            w_cbd = 0.55 + 0.10*rng.random()     # 0.55~0.65
        else:
            w_cbd = 0.75 + 0.10*rng.random()     # 0.75~0.85（
        w_non = 1.0 - w_cbd

        cbd_candidates = np.where(cbd_sink_mask)[0]
        non_candidates = np.where(non_cbd_sink_mask)[0]

        if len(cbd_candidates) > 0:
            x = rng.random(len(cbd_candidates))
            x = x / x.sum()
            dest[i, cbd_candidates] = w_cbd * x
        if len(non_candidates) > 0:
            y = rng.random(len(non_candidates))
            y = y / y.sum()
            dest[i, non_candidates] = w_non * y


    dest = dest / dest.sum(axis=1, keepdims=True)

    omega = compute_expected_time(dest, c_time)
    lambda_vec = np.where(is_cbd_link, 120.0, 25.0)  
    '''
    lambda_vec = np.where(
        is_cbd_link,
        np.random.normal(loc=120, scale=30, size=n_links),  
        np.random.normal(loc=25,  scale=10, size=n_links)   
    )
    lambda_vec = np.clip(lambda_vec, 5, None)  
    '''



    v = initial_charge + unit_fare * c
    v = np.round(v, 2)


    mu0 = np.ones(L) / L
    features = [lambda_vec.copy()]  
    init_m = np.full(L, 0.2, dtype=np.float32)
    init_mu = np.full(L, 1.0 / L, dtype=np.float32)

    params = dict(
        alpha=alpha,
        M=M,
        b=b,
        initial_charge=initial_charge,
        unit_fare=unit_fare,
        speed_mph=speed_mph
    )


    link_index = np.arange(L)
    link_tail = tail_of
    link_head = head_of

    return {

        "c": c,
        "c_time": c_time,
        "tau": tau,
        "outgoing_dict": outgoing_dict,
        "outgoing_links": outgoing_links,
        "v": v,
        "dest": dest,
        "omega": omega,
        "lambda_vec": lambda_vec,
        "features": features,
        "mu0": mu0,
        "init_m": init_m,
        "init_mu": init_mu,
        "M": M,
        "b": b,
        "alpha": alpha,
        "n_links": L,
        "directed_links": directed_links,  # list of (u->v)
        "link_tail": link_tail,
        "link_head": link_head,
        "coords": coords,
        "params": params,
    }





In [6]:


def softmax_grad_wrt_theta_r(pi, x_r, outgoing_mask):

    exp_x = (pi * x_r).sum(axis=1, keepdims=True)  # (n,1)
    # ∂π = π ⊙ (x_r - E_pi[x_r])
    dpi = pi * (x_r - exp_x)
    # keep only valid (s,a)
    dpi = np.where(outgoing_mask, dpi, 0.0)
    return dpi

def build_DM_matrix(Q, pi):

    return Q - pi

def build_A_B_m_system(lambda_vec, tau, omega, M, alpha, mu, m):

    n = len(mu)
    w = np.dot(mu, tau + m * omega)  # scalar
    
    safe_mu = np.where(mu > 0, mu, 1.0)
    z = alpha * lambda_vec * w / (M * safe_mu)
    h = np.exp(-z) * alpha * lambda_vec / M  # (n,)

    # A_su = δ_su - h_s * (μ_u ω_u)/μ_s
    A = np.eye(n) - np.outer(h / safe_mu, mu * omega)

    # B_su = - h_s * (τ_u + m_u ω_u)/μ_s, then add diagonal + h_s * (w/μ_s^2)
    B = - np.outer(h / safe_mu, (tau + m * omega))
    B[np.arange(n), np.arange(n)] += h * (w / (safe_mu**2))
    return A, B, h, w

def stationary_sensitivity_rhs(mu, m, dpi_dtheta_r):

    weight = (mu * (1.0 - m))[:, None]  # (n,1)
    # dpi_dtheta_r is (n,n) with rows v, cols u
    b_pi_row = (weight * dpi_dtheta_r).sum(axis=0)  # shape (n,)
    return b_pi_row  # as row; we’ll convert to column when solving

def build_P_row_scaled(pi, Q, m):

    import numpy as np
    try:
        from scipy.sparse import issparse, diags
    except Exception:
        issparse = lambda x: False

    n = m.shape[0]
    if issparse(pi) or issparse(Q):

        assert pi.shape == (n, n) and Q.shape == (n, n)
        D1 = diags(1.0 - m, offsets=0, shape=(n, n))
        D2 = diags(m,         offsets=0, shape=(n, n))
        return D1 @ pi + D2 @ Q
    else:

        return (1.0 - m)[:, None] * pi + m[:, None] * Q


def solve_sensitivities_single_param(
    lambda_vec, tau, omega, M, alpha,
    Q, pi, mu, m,
    x_r, outgoing_mask,
    enforce_sum_zero_row=0,
):

    n = len(mu)
    mu = mu.reshape(-1)  # 1D
    m  = m.reshape(-1)


    A, B, h, w = build_A_B_m_system(lambda_vec, tau, omega, M, alpha, mu, m)


    
    P = build_P_row_scaled(pi, Q, m)



    if issparse(pi) or issparse(Q):
        if not issparse(pi): pi = csr_matrix(pi)
        if not issparse(Q):  Q  = csr_matrix(Q)
        D  = Q - pi
        Bm = diags(mu) @ D                 
    else:
        pi = np.asarray(pi)
        Q  = np.asarray(Q)
        D  = np.asarray(Q - pi)            
        Bm = (mu.reshape(-1, 1)) * D       


    dpi = softmax_grad_wrt_theta_r(pi, x_r, outgoing_mask)  # (n,n)


    b_pi = stationary_sensitivity_rhs(mu, m, dpi).reshape(-1, 1)  # (n,1)


    X = solve(A, B)                    # solves A X = B  → X = A^{-1} B
    K = Bm.T @ X                       
    M_sys = np.eye(n) - P.T + K        

    rhs = b_pi.copy()                  # (n,1)
    row = enforce_sum_zero_row
    M_sys[row, :] = 1.0
    rhs[row, 0] = 0.0


    dmu = solve(M_sys, rhs).reshape(-1)     # (n,)


    dm = - solve(A, B @ dmu)                # (n,)


    res1 = A @ dm + B @ dmu                 # should be ≈ 0
    res2 = (np.eye(n) - P.T) @ dmu - (Bm.T @ dm + b_pi.reshape(-1))  # should be ≈ 0

    diagnostics = dict(
        res_A_norm = np.linalg.norm(res1, ord=np.inf),
        res_stationarity_norm = np.linalg.norm(res2, ord=np.inf),
        M_sys_norm = np.linalg.norm(M_sys, ord=2),
    )
    

    sA = np.linalg.svd(A, compute_uv=False)
    condA = sA.max() / sA.min()
    print('cond(A)=', condA)



    M_aug = M_sys.copy()
    row = 0
    M_aug[row,:] = 1.0
    s = np.linalg.svd(M_aug, compute_uv=False)
    condM = s.max() / s.min()
    print('sigma_min(M_aug)=', s.min(), '  cond(M_aug)=', condM)
    return dmu, dm, diagnostics


In [7]:
### Gradient-Based BFGS


current_state = {}

def make_wrapped_objective_BFGS(
    outgoing_dict,
    action_features,
    lambda_vec,
    tau,
    omega,
    M,
    alpha,
    Q,        
    v,
    c,
    beta,
    init_mu
):
    n = len(lambda_vec)             
    mu_last = init_mu.copy()
    m_last = np.full(n, 0.2, dtype=np.float32)

    theta_hist, R_hist, m_hist, mu_hist = [], [], [], []

    row_idx, col_idx = extract_state_action_pairs(outgoing_dict)
    n_links = len(lambda_vec)

    def objective_wrapper(theta_vec):
        nonlocal mu_last, m_last
        pi = compute_softmax_policy_by_action_features(
            action_features, theta_vec, n_links, row_idx, col_idx
        )

        pi_dense = pi.toarray() if issparse(pi) else np.asarray(pi)

        mu, m = compute_joint_fixed_point_single_loop(
            lambda_vec, tau, omega, M, alpha, Q, outgoing_dict, pi,
            mu_last, m_last, row_idx, col_idx, v, c,
            tol_m=1e-3, tol_mu=1e-4, max_iter=6000, return_history=False
        )


        kappa = compute_kappa(v, c, Q, beta)

        R_pi = compute_R(mu, tau, omega, m, kappa, beta)


        N = np.sum(mu * (-beta * tau + kappa * m))
        D = np.sum(mu * (tau + omega * m))


        dN_dm  = mu * kappa
        dN_dmu = -beta * tau + kappa * m
        dD_dm  = mu * omega
        dD_dmu = tau + omega * m


        global current_state
        current_state = {
            'mu': mu,
            'm': m,
            'pi': pi_dense,
            'Q': Q,                 
            'kappa': kappa,
            'N': N, 'D': D,
            'dN_dm': dN_dm, 'dN_dmu': dN_dmu,
            'dD_dm': dD_dm, 'dD_dmu': dD_dmu,
            'R': R_pi
        }

  
        theta_hist.append(theta_vec.copy())
        R_hist.append(R_pi)
        m_hist.append(m.copy())
        mu_hist.append(mu.copy())
        print(f"[Step] theta = {theta_vec},  R(pi) = {R_pi:.6f}")

        return -R_pi

    return objective_wrapper, theta_hist, R_hist, m_hist, mu_hist



class ObjectiveWithGrad:
    def __init__(self,
                 wrapped_objective,     
                 lambda_vec, tau, omega, beta,
                 M, alpha,
                 x_r, outgoing_mask,    
                 solve_sensitivities_single_param):
        self.fun = wrapped_objective
        self.lambda_vec = lambda_vec
        self.tau = tau
        self.omega = omega
        self.beta = beta
        self.M = M
        self.alpha = alpha
        self.x_r = x_r
        self.outgoing_mask = outgoing_mask
        self.solver = solve_sensitivities_single_param

    def objective(self, theta):
        return self.fun(theta)

    def grad(self, theta):
        if not current_state:
            _ = self.fun(theta)

        st = current_state
        mu = st['mu']; m = st['m']
        pi = st['pi']; Q = st['Q']
        N, D = st['N'], st['D']
        dN_dm, dN_dmu = st['dN_dm'], st['dN_dmu']
        dD_dm, dD_dmu = st['dD_dm'], st['dD_dmu']

        dmu, dm, info = self.solver(
            self.lambda_vec, self.tau, self.omega, self.M, self.alpha,
            Q, pi, mu, m,
            self.x_r, self.outgoing_mask,
            enforce_sum_zero_row=0
        )

        term_N = float(np.dot(dN_dm, dm) + np.dot(dN_dmu, dmu))
        term_D = float(np.dot(dD_dm, dm) + np.dot(dD_dmu, dmu))
        grad_R = (term_N * D - N * term_D) / (D ** 2)

        return np.array([-grad_R])




In [8]:
net = build_sioux_falls_like(seed=2025)

print("n_links:", net["n_links"])                 # 76
print("c shape:", net["c"].shape)                 # (76, 76)
print("tau shape:", net["tau"].shape)             # (76,)
print("outgoing example (link 0):", net["outgoing_dict"][0])
print("dest row sums (min/max):", net["dest"].sum(axis=1).min(), net["dest"].sum(axis=1).max())
print("lambda_vec (CBD sample):", net["lambda_vec"][:10])

omega = compute_expected_time(net["dest"], net["c_time"])
omega

n_links: 76
c shape: (76, 76)
tau shape: (76,)
outgoing example (link 0): [0, 3, 43]
dest row sums (min/max): 0.9999999999999998 1.0000000000000002
lambda_vec (CBD sample): [25. 25. 25. 25. 25. 25. 25. 25. 25. 25.]


array([0.14353345, 0.18271664, 0.12148608, 0.14263611, 0.13012773,
       0.12185846, 0.1463575 , 0.11823776, 0.17758063, 0.14501631,
       0.12510062, 0.15374641, 0.10959962, 0.13128566, 0.10942111,
       0.10275911, 0.12340849, 0.11017524, 0.15316889, 0.12305347,
       0.12519599, 0.15675447, 0.10547489, 0.12759949, 0.10138943,
       0.10509104, 0.12743364, 0.10766232, 0.15906343, 0.11981964,
       0.14531532, 0.18642482, 0.11616445, 0.15469907, 0.11936353,
       0.12932314, 0.14546256, 0.12236808, 0.17633287, 0.1578764 ,
       0.1541027 , 0.17815043, 0.11558452, 0.14582459, 0.10650447,
       0.12804942, 0.10216065, 0.13114594, 0.11102174, 0.15025563,
       0.15719643, 0.17517273, 0.15296992, 0.15676348, 0.11589272,
       0.12582767, 0.10553693, 0.10400432, 0.10040061, 0.10896133,
       0.12742643, 0.12281097, 0.15238632, 0.14591067, 0.18155797,
       0.16210847, 0.1474999 , 0.11954102, 0.13554932, 0.10707222,
       0.12771348, 0.10958569, 0.14710958, 0.12222671, 0.17687

In [9]:
M = 1000
alpha = 0.8
n_links = len(net["lambda_vec"])

init_m = np.full(n_links, 0.2, dtype=np.float32)
init_mu = np.full(n_links, 1/n_links, dtype=np.float32)

row_idx, col_idx = extract_state_action_pairs(net["outgoing_dict"])

features = [net["lambda_vec"]]



theta_vec = [0.2]
# 计算 policy
pi = compute_softmax_policy_by_action_features(
    features, theta_vec, n_links, row_idx, col_idx
)
pi

<76x76 sparse matrix of type '<class 'numpy.float64'>'
	with 252 stored elements in Compressed Sparse Row format>

In [10]:
initial_charge = 35  
unit_fare = 35     # per mile

gamma = 2.5*40  # fare
beta = 0.6*40   # unit cost


v = initial_charge + unit_fare * net["c"]

v = np.round(v, 2)
v

array([[150.39,  89.55,  87.96, ..., 383.65, 395.91, 438.8 ],
       [ 95.84, 150.39, 148.8 , ..., 437.36, 449.62, 492.51],
       [198.65, 137.82, 136.22, ..., 330.69, 342.96, 385.84],
       ...,
       [401.57, 343.9 , 342.31, ..., 127.66, 142.75, 182.81],
       [499.03, 439.42, 437.83, ..., 132.46, 132.98,  82.47],
       [452.79, 391.95, 390.36, ..., 182.97,  85.51, 132.98]])

In [11]:
mu1, m1 = compute_joint_fixed_point_single_loop(
    net["lambda_vec"], net["tau"], omega, M, alpha, net["dest"], net["outgoing_dict"], pi, init_mu, init_m, row_idx, col_idx,v,net["c"],
    tol_m=1e-3, tol_mu=1e-4, max_iter=2000, return_history=False
)

ite:0,m_diff: 5.3017e-01,  mu_norm: 3.0166e-01, mu_diff:2.9890e-02, R:84.54151507605533
Converged! Iteration:274, Duration:0.039975881576538086
m_diff: 3.5508e-05,  mu_norm: 9.9721e-05, mu_diff:2.4937e-05
min m:0.094633835479907, 0.5:60,0.8:17, 0.9:8,0.99:7


In [12]:
### Derivative-free method: Nelder-Mead

n = 76

wrapped_objective, theta_hist, R_hist, m_hist, mu_hist = make_wrapped_objective(
    net["outgoing_dict"],
    features,
    net["lambda_vec"],
    net["tau"],
    omega,
    M,
    alpha,
    net["dest"],
    v,
    net["c"],
    beta,
    init_mu
)


t1 = time.time()
result = minimize(
    wrapped_objective,
    x0=np.array([0]),
    method='Nelder-Mead',
    options={
        'disp': True,
        'maxiter': 1000,       
        'adaptive': True       
    }
)



t2 = time.time()

print(t2-t1)


theta:[0.]
ite:0,m_diff: 5.3017e-01,  mu_norm: 1.0522e-01, mu_diff:7.9643e-03, R:84.54151507605533
Converged! Iteration:266, Duration:0.027353763580322266
m_diff: 3.4650e-05,  mu_norm: 9.9631e-05, mu_diff:2.4914e-05
min m:0.09648872909453224, 0.5:60,0.8:9, 0.9:8,0.99:8
[Step] theta = [0.000000], R(pi) = 171.357117

theta:[0.00025]
ite:0,m_diff: 5.3017e-01,  mu_norm: 1.0522e-01, mu_diff:7.9643e-03, R:84.54151507605533
Converged! Iteration:266, Duration:0.02643108367919922
m_diff: 3.4677e-05,  mu_norm: 9.9688e-05, mu_diff:2.4928e-05
min m:0.09647013575725748, 0.5:60,0.8:9, 0.9:8,0.99:8
[Step] theta = [0.000250], R(pi) = 171.368266

theta:[0.0005]
ite:0,m_diff: 5.3017e-01,  mu_norm: 1.0522e-01, mu_diff:7.9643e-03, R:84.54151507605533
Converged! Iteration:266, Duration:0.02538609504699707
m_diff: 3.4704e-05,  mu_norm: 9.9746e-05, mu_diff:2.4943e-05
min m:0.0964515759663685, 0.5:60,0.8:9, 0.9:8,0.99:8
[Step] theta = [0.000500], R(pi) = 171.379413

theta:[0.001]
ite:0,m_diff: 5.3017e-01,  m

In [13]:

outgoing_mask = np.zeros((n, n), dtype=bool)
for s, actions in net["outgoing_dict"].items():
    outgoing_mask[s, actions] = True
    
n_features = 2
action_features = np.zeros((n_features, n, n))


for s, actions in net["outgoing_dict"].items():
    for a in actions:
        action_features[0, s, a] = net["lambda_vec"][a]   


x_r = action_features[0]


if issparse(pi): 
    pi = pi.toarray()
    
Q = net["dest"].copy()

if issparse(Q):  
    Q = Q.toarray()

dmu, dm, info = solve_sensitivities_single_param(
    net["lambda_vec"], net["tau"], omega, M, alpha,
    Q, pi, mu1, m1,
    x_r, outgoing_mask,
    enforce_sum_zero_row=0,  
)
print(info)  

cond(A)= 2.315602705388983
sigma_min(M_aug)= 0.0054575469114326695   cond(M_aug)= 1600.874382200497
{'res_A_norm': 7.940933880509066e-23, 'res_stationarity_norm': 4.567809025115826e-18, 'M_sys_norm': 8.736847040170005}


In [14]:

wrapped_objective, theta_hist, R_hist, m_hist, mu_hist = make_wrapped_objective_BFGS(
    outgoing_dict=net["outgoing_dict"],
    action_features=features,          # 例：features=[lambda_vec] 或更多
    lambda_vec=net["lambda_vec"],
    tau=net["tau"],
    omega=omega,
    M=M,
    alpha=alpha,
    Q=net["dest"],
    v=v,
    c=net["c"],
    beta=beta,
    init_mu=init_mu
)



obj = ObjectiveWithGrad(
    wrapped_objective=wrapped_objective,
    lambda_vec=net["lambda_vec"],
    tau=net["tau"],
    omega=omega,
    beta=beta,
    M=M,
    alpha=alpha,
    x_r=x_r,
    outgoing_mask=outgoing_mask,
    solve_sensitivities_single_param=solve_sensitivities_single_param
)


# 3) 跑 BFGS

t1 = time.time()
res = minimize(
    fun=obj.objective,
    jac=obj.grad,
    x0=np.array([0.1]),
    method="BFGS",                    
    options=dict(disp=True, maxiter=500, gtol=1e-8)
)
print(res)
t2 = time.time()

print(t2-t1)




ite:0,m_diff: 5.3017e-01,  mu_norm: 3.0161e-01, mu_diff:2.9881e-02, R:84.54151507605533
Converged! Iteration:274, Duration:0.02899622917175293
m_diff: 3.5508e-05,  mu_norm: 9.9720e-05, mu_diff:2.4936e-05
min m:0.094634122125748, 0.5:60,0.8:17, 0.9:8,0.99:7
[Step] theta = [0.1],  R(pi) = 172.363363
cond(A)= 2.315612839222503
sigma_min(M_aug)= 0.005457574287331492   cond(M_aug)= 1600.8657247460237
ite:0,m_diff: 5.3017e-01,  mu_norm: 3.0165e-01, mu_diff:2.9888e-02, R:84.54151507605533
Converged! Iteration:274, Duration:0.026996850967407227
m_diff: 3.5508e-05,  mu_norm: 9.9721e-05, mu_diff:2.4937e-05
min m:0.09463389349775533, 0.5:60,0.8:17, 0.9:8,0.99:7
[Step] theta = [0.11681385],  R(pi) = 172.363506
cond(A)= 2.3156047565471662
sigma_min(M_aug)= 0.005457552452291536   cond(M_aug)= 1600.8726299149241
ite:0,m_diff: 5.3017e-01,  mu_norm: 3.0166e-01, mu_diff:2.9889e-02, R:84.54151507605533
Converged! Iteration:274, Duration:0.026664257049560547
m_diff: 3.5508e-05,  mu_norm: 9.9721e-05, mu_di

Converged! Iteration:274, Duration:0.026459217071533203
m_diff: 3.5508e-05,  mu_norm: 9.9721e-05, mu_diff:2.4937e-05
min m:0.09463383545858187, 0.5:60,0.8:17, 0.9:8,0.99:7
[Step] theta = [0.25332021],  R(pi) = 172.363542
cond(A)= 2.315602704635048
sigma_min(M_aug)= 0.00545754690939594   cond(M_aug)= 1600.8743828446088
Optimization terminated successfully.
         Current function value: -172.363542
         Iterations: 20
         Function evaluations: 21
         Gradient evaluations: 21
      fun: -172.3635419001098
 hess_inv: array([[459181.85392274]])
      jac: array([-7.94492864e-09])
  message: 'Optimization terminated successfully.'
     nfev: 21
      nit: 20
     njev: 21
   status: 0
  success: True
        x: array([0.25332021])
0.8597042560577393


In [15]:
### Numerical BFGS

t3 = time.time()
res = minimize(
    fun=obj.objective,          
    x0=np.array([0.1]),         
    method="BFGS",              
    options=dict(disp=True, maxiter=500, gtol=1e-8)
)
t4 = time.time()

print(res)

print(t4-t3)

ite:0,m_diff: 5.3017e-01,  mu_norm: 3.0161e-01, mu_diff:2.9881e-02, R:84.54151507605533
Converged! Iteration:274, Duration:0.029837846755981445
m_diff: 3.5508e-05,  mu_norm: 9.9720e-05, mu_diff:2.4936e-05
min m:0.094634122125748, 0.5:60,0.8:17, 0.9:8,0.99:7
[Step] theta = [0.1],  R(pi) = 172.363363
ite:0,m_diff: 5.3017e-01,  mu_norm: 3.0161e-01, mu_diff:2.9881e-02, R:84.54151507605533
Converged! Iteration:274, Duration:0.02732706069946289
m_diff: 3.5508e-05,  mu_norm: 9.9720e-05, mu_diff:2.4936e-05
min m:0.09463412212534242, 0.5:60,0.8:17, 0.9:8,0.99:7
[Step] theta = [0.10000001],  R(pi) = 172.363363
ite:0,m_diff: 5.3017e-01,  mu_norm: 3.0165e-01, mu_diff:2.9888e-02, R:84.54151507605533
Converged! Iteration:274, Duration:0.024946928024291992
m_diff: 3.5508e-05,  mu_norm: 9.9721e-05, mu_diff:2.4937e-05
min m:0.09463389274860992, 0.5:60,0.8:17, 0.9:8,0.99:7
[Step] theta = [0.11695061],  R(pi) = 172.363506
ite:0,m_diff: 5.3017e-01,  mu_norm: 3.0165e-01, mu_diff:2.9888e-02, R:84.5415150760

Converged! Iteration:274, Duration:0.02506113052368164
m_diff: 3.5508e-05,  mu_norm: 9.9721e-05, mu_diff:2.4937e-05
min m:0.09463383546302831, 0.5:60,0.8:17, 0.9:8,0.99:7
[Step] theta = [0.21625335],  R(pi) = 172.363542
ite:0,m_diff: 5.3017e-01,  mu_norm: 3.0166e-01, mu_diff:2.9890e-02, R:84.54151507605533
Converged! Iteration:274, Duration:0.025045394897460938
m_diff: 3.5508e-05,  mu_norm: 9.9721e-05, mu_diff:2.4937e-05
min m:0.09463383546232598, 0.5:60,0.8:17, 0.9:8,0.99:7
[Step] theta = [0.21800525],  R(pi) = 172.363542
ite:0,m_diff: 5.3017e-01,  mu_norm: 3.0166e-01, mu_diff:2.9890e-02, R:84.54151507605533
Converged! Iteration:274, Duration:0.02453470230102539
m_diff: 3.5508e-05,  mu_norm: 9.9721e-05, mu_diff:2.4937e-05
min m:0.09463383546232598, 0.5:60,0.8:17, 0.9:8,0.99:7
[Step] theta = [0.21800527],  R(pi) = 172.363542
ite:0,m_diff: 5.3017e-01,  mu_norm: 3.0166e-01, mu_diff:2.9890e-02, R:84.54151507605533
Converged! Iteration:274, Duration:0.024671077728271484
m_diff: 3.5508e-05,

Converged! Iteration:274, Duration:0.026797056198120117
m_diff: 3.5508e-05,  mu_norm: 9.9721e-05, mu_diff:2.4937e-05
min m:0.09463383546209905, 0.5:60,0.8:17, 0.9:8,0.99:7
[Step] theta = [0.21863939],  R(pi) = 172.363542
ite:0,m_diff: 5.3017e-01,  mu_norm: 3.0166e-01, mu_diff:2.9890e-02, R:84.54151507605533
Converged! Iteration:274, Duration:0.026058197021484375
m_diff: 3.5508e-05,  mu_norm: 9.9721e-05, mu_diff:2.4937e-05
min m:0.09463383546209916, 0.5:60,0.8:17, 0.9:8,0.99:7
[Step] theta = [0.21863938],  R(pi) = 172.363542
ite:0,m_diff: 5.3017e-01,  mu_norm: 3.0166e-01, mu_diff:2.9890e-02, R:84.54151507605533
Converged! Iteration:274, Duration:0.026087045669555664
m_diff: 3.5508e-05,  mu_norm: 9.9721e-05, mu_diff:2.4937e-05
min m:0.09463383546209905, 0.5:60,0.8:17, 0.9:8,0.99:7
[Step] theta = [0.21863939],  R(pi) = 172.363542
ite:0,m_diff: 5.3017e-01,  mu_norm: 3.0166e-01, mu_diff:2.9890e-02, R:84.54151507605533
Converged! Iteration:274, Duration:0.026020050048828125
m_diff: 3.5508e-0

In [16]:
### package 检查gradient after optimization

from scipy.optimize import check_grad
import numpy as np


def fun_wrapped(theta_vec):
    return obj.objective(theta_vec)


def grad_wrapped(theta_vec):
    return obj.grad(theta_vec)

x0 = np.array([0.1])      
err = check_grad(fun_wrapped, grad_wrapped, x0)
print("check_grad error:", err)


cond(A)= 2.3156027047594
sigma_min(M_aug)= 0.005457546909732139   cond(M_aug)= 1600.8743827382916
ite:0,m_diff: 5.3017e-01,  mu_norm: 3.0161e-01, mu_diff:2.9881e-02, R:84.54151507605533
Converged! Iteration:274, Duration:0.028058290481567383
m_diff: 3.5508e-05,  mu_norm: 9.9720e-05, mu_diff:2.4936e-05
min m:0.094634122125748, 0.5:60,0.8:17, 0.9:8,0.99:7
[Step] theta = [0.1],  R(pi) = 172.363363
ite:0,m_diff: 5.3017e-01,  mu_norm: 3.0161e-01, mu_diff:2.9881e-02, R:84.54151507605533
Converged! Iteration:274, Duration:0.027980804443359375
m_diff: 3.5508e-05,  mu_norm: 9.9720e-05, mu_diff:2.4936e-05
min m:0.09463412212534242, 0.5:60,0.8:17, 0.9:8,0.99:7
[Step] theta = [0.10000001],  R(pi) = 172.363363
check_grad error: 0.01695039303690773


In [ ]:
##### end